In [ ]:
import pandas as pd
import glob
import os

In [ ]:
# Define the file path
file_path =  r'C:/Users/aparn/Box/InTrans/RWRAD_Internal/Final_files_with_variables/LOSO_CV results/losocv_results_with_hyperparameter_tuning_parallel_demo_sleep.csv'

# Read the CSV file into a DataFrame
all_var_df= pd.read_csv(file_path)

In [ ]:
all_var_df.head(5)

In [ ]:
all_var_df['Subject'].nunique()

In [ ]:
# Assuming the DataFrame is named `df` and the relevant columns are 'subj' for the subject and 'True_Label' for the label

# Group by 'subj' (subject) and check unique values in 'True_Label' for each subject
unique_label_counts = all_var_df.groupby('Subject')['True Label'].nunique()

# Count how many subjects have more than one unique value for 'True_Label'
subjects_with_multiple_labels = (unique_label_counts > 1).sum()

# Print the result
print(f"Number of subjects with more than 1 unique value for True Label: {subjects_with_multiple_labels}")


In [ ]:
# Assuming the DataFrame is named `df` and the relevant columns are 'subj' for the subject and 'True_Label' for the label

# Group by 'subj' and check how many unique values exist for 'True_Label'
unique_label_counts = all_var_df.groupby('Subject')['True Label'].nunique()

# Get the subjects where the number of unique labels is greater than 1
subjects_with_multiple_labels = unique_label_counts[unique_label_counts > 1].index

# Print the subjects with more than 1 unique value for True Label
print("Subjects with more than 1 unique value for True Label:")
print(subjects_with_multiple_labels)


In [ ]:
import pandas as pd

# Assuming the DataFrame is named `df` with columns 'subj', 'True_Label', and 'Predicted_Label'

# Group by 'subj' to summarize data for each subject
summary_df = all_var_df.groupby('Subject').agg(
    num_data_points=('Subject', 'size'),
    true_label=('True Label', lambda x: x.iloc[0] if len(x.unique()) == 1 else 'Multiple'),  # Repeat or mark 'Multiple' if more than 1 unique True_Label
    num_predicted_1=('Predicted Label', lambda x: (x == 1).sum()),  # Count of Predicted_Label == 1
    num_predicted_0=('Predicted Label', lambda x: (x == 0).sum())  # Count of Predicted_Label == 0
)

# Add a column for Final_call: 1 if num_predicted_1 > num_predicted_0, otherwise 0
summary_df['Final_call'] = (summary_df['num_predicted_1'] > summary_df['num_predicted_0']).astype(int)

# Reset index to make 'subj' a regular column again (for easier viewing)
summary_df.reset_index(inplace=True)



In [ ]:
summary_df.head(65)

In [ ]:
len(summary_df)

In [ ]:
import pandas as pd

# Assuming `expanded_summary_df` is your DataFrame
# Remove rows where the 'true_label' is "Multiple"
filtered_df = summary_df[summary_df['true_label'] != 'Multiple']

# Create a new column to check if True_Label matches Final_Call
filtered_df['correct_prediction'] = filtered_df['true_label'] == filtered_df['Final_call']

# Calculate accuracy by dividing the number of correct predictions by the total number of data points
accuracy = filtered_df['correct_prediction'].sum() / len(filtered_df)

# Calculate the count of correct predictions
correct_prediction_count = filtered_df['correct_prediction'].sum()

# Print the accuracy and the count of correct predictions
print(f"Accuracy: {accuracy:.4f}")
print(f"Count of Correct Predictions: {correct_prediction_count}")


In [ ]:
filtered_df.head()

In [ ]:
import pandas as pd

# Create variables for True Positives, False Positives, False Negatives, and True Negatives
tp = ((filtered_df['true_label'] == 1) & (filtered_df['Final_call'] == 1)).sum()
fp = ((filtered_df['true_label'] == 0) & (filtered_df['Final_call'] == 1)).sum()
fn = ((filtered_df['true_label'] == 1) & (filtered_df['Final_call'] == 0)).sum()
tn = ((filtered_df['true_label'] == 0) & (filtered_df['Final_call'] == 0)).sum()

# Calculate Precision, Recall, and F1 Score
precision = tp / (tp + fp) if (tp + fp) != 0 else 0  # Avoid division by zero
recall = tp / (tp + fn) if (tp + fn) != 0 else 0  # Avoid division by zero
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0  # Avoid division by zero

# Print the results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")